In [2]:
import numpy as np 
import tensorflow as tf
import pickle 
from tensorflow.keras import layers ,activations,models,preprocessing

## PREPROCESSING THE DATA


In [3]:
import requests,zipfile,io

r = requests.get('https://github.com/shubham0204/Dataset_Archives/blob/master/chatbot_nlp.zip?raw=true')
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

yaml filelarını parsela <br>

2 veya daha fazla cümleyi birleştri eğer cevap 2 veya daha fazla ise <br>
istenmeyen tipteki dataları kaldır <br>
Tüm cevaplara START ve END taglari ekle <br>
Tokenizer oluşturup tüm sözlüğü questin + answer şekline getir

In [36]:
from tensorflow.keras import preprocessing , utils
import os
import yaml

dir_path = 'chatbot_nlp/data'
files_list = os.listdir(dir_path + os.sep)

questions = list()
answers = list()

## yaml file tek tek okumak için
for filepath in files_list:
    stream = open( dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(stream)
    conversations = docs['conversations']
    for con in conversations:
        ## eğer 2 den fazla cevap varsa bunları birbirine ekle 
        if len( con ) > 2 :
            questions.append(con[0])
            replies = con[ 1 : ]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append( ans )
            ## tek cevap varsa direkt answerslara ekle 
        elif len( con )> 1:
            questions.append(con[0])
            answers.append(con[1])

answers_with_tags = list()
# tagleri ekleyebilmek için öncelikle tip kontrolu yap 
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )

answers = list() 
## her bir textin başına ve sonuna tagler ekle
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

VOCAB SIZE : 1894


## Preparing data for Seq2Seq Model
3 tip data gerekmekte ; <br>
encoder_input_data = Soruları tokenize et ve maksimum len ayarla <br>
decoder_input_data = cevapları tokenize et ve maksimum len ayarla <br>
decoder_output_data = cevapları tokenize et ve tüm tokenized answerlardan START tagini kaldır


In [70]:
from gensim.models import Word2Vec
import re 

vocab = []
for word in tokenizer.word_index:
    vocab.append(word)
    
def tokenize(sentences):
    token_list = []
    vocabulary = []
    
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub("[^a-zA-z]"," ",sentence)
        tokens = sentence.split()
        vocabulary += tokens
        token_list.append(tokens)
        
    return token_list,vocabulary

p = tokenize(questions+answers)
model = Word2Vec(p[0])

embedding_matrix = np.zeros((VOCAB_SIZE,100))
for i in range(len(tokenizer.word_index)):
    try : 
        embedding_matrix[i] = model[vocab[i]]
    except :
        pass
## encoder input data 
tokenized_questions = tokenizer.texts_to_sequences(questions)
maxlen_questions = max([len(i) for i in tokenized_questions])
padded_questions = preprocessing.sequence.pad_sequences(tokenized_questions,maxlen=maxlen_questions, padding="post")
encoder_input_data = np.array(padded_questions)
print(encoder_input_data.shape,maxlen_questions)


## decoder input data 
tokenized_answers = tokenizer.texts_to_sequences(answers)
maxlen_answers = max([len(i) for i in tokenized_answers])
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers,maxlen=maxlen_answers,padding="post")
decoder_input_data = np.array(padded_answers)
print(decoder_input_data.shape,maxlen_answers)

## decoder output data 
tokenized_answers = tokenizer.texts_to_sequences(answers)
for i in range(len(tokenized_answers)):
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers,maxlen=maxlen_answers,padding="post")
onehot_answers = utils.to_categorical(padded_answers,VOCAB_SIZE)
decoder_output_data = np.array(onehot_answers)
print(decoder_output_data.shape)

(564, 22) 22
(564, 74) 74
(564, 74, 1894)


C:\Users\bagat\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


## Define Encoder-Decoder model 
2 input layer kullanılacak. 1 tane encoder_input_data için bir tane decoder_input_data için <br>
Embedding Layer : token vecktörlerini dense vektörlere çevirmek için <br>

1. encoder_input_data embedding layera gelecek. 
2. Embedding layerdan çıkan çıktı lstm e gidicek ve lstm 2 state vector üretecek (h,c)
3. decoder_input_data sı embedding layerdan gelicek 
4. sequencesler üretilecek 

In [95]:

encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, None, 200)    378800      input_9[0][0]                    
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, None, 200)    378800      input_10[0][0]                   
____________________________________________________________________________________________

In [96]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=5 ) 
model.save( 'model.h5' ) 


Epoch 1/5
564/564 [==============================] - 7s 12ms/sample - loss: 1.3054
Epoch 2/5
564/564 [==============================] - 5s 9ms/sample - loss: 1.1165
Epoch 3/5
564/564 [==============================] - 5s 9ms/sample - loss: 1.0961
Epoch 4/5
564/564 [==============================] - 5s 9ms/sample - loss: 1.0792
Epoch 5/5
564/564 [==============================] - 5s 9ms/sample - loss: 1.0618


## Defining inference models 
Cevapları tahmin etmek için inference model kullanabilirizi 

Encoder inference model = Soruyu alır ve lstm stateleri üretir <br>
Decoder inference model = 2 input alır biri encoder stateleri, diğeri input sequencesin cevapları 

In [97]:

def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model
    

## Talking with chatbot
str_to_token  str olarak sorulan soruları tokenlere çevirmek için

In [102]:
def str_to_tokens(sentence : str):
    words = sentence.lower().split()
    token_list = list()
    
    for word in words:
        token_list.append(tokenizer.word_index[word])
        
    return preprocessing.sequence.pad_sequences([token_list],maxlen=maxlen_questions,padding="post")

1. First, we take a question as input and predict the state values using enc_model.
1. We set the state values in the decoder's LSTM.
1. Then, we generate a sequence which contains the <start> element.
1. We input this sequence in the dec_model.
1. We replace the <start> element with the element which was predicted by the dec_model and update the state values.
1. We carry out the above steps iteratively till we hit the <end> tag or the maximum answer length.1. 

In [105]:

enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )

Enter question : why
 i i end
Enter question : why why 
 i i end
Enter question : sdasd


KeyError: 'sdasd'